In [1]:
# !pip install --upgrade pip
# !pip install matplotlib
# !pip install -U scikit-learn
# !pip install sktime
# !pip install torch torchvision torchaudio
# !pip install transformers
# !pip install pytesseract
# !pip install opencv-python

In [2]:
from core import data_manager, DataManager, settings
from core.utils import setup_logging
from pathlib import Path

settings.logging.level = "DEBUG"

setup_logging()

In [3]:
# data_manager.backup_data(paths=[Path("/Users/igor/Desktop/App-parser/data/raw/")])

In [4]:
coins = {}

for coin in data_manager.coin_list:
    if coin != "TON":
        continue
    for path in data_manager.get_path(data_type="raw",
                                      coin=coin,
                                      timetravel="5m"):
        coins.setdefault(coin, [])
        coins[coin].append(path)

In [5]:
from backend.Dataset import DatasetTimeseries
import pandas as pd

/Users/igor/Library/Caches/pypoetry/virtualenvs/postgresdocker-MncL8gKv-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
coins

{'TON': [PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_1/TON/TON_5m.csv'),
  PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_13/TON/TON_5m.csv'),
  PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_15/TON/TON_5m.csv'),
  PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_5/TON/TON_5m.csv'),
  PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_2/TON/TON_5m.csv'),
  PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_3/TON/TON_5m.csv'),
  PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_4/TON/TON_5m.csv')]}

In [7]:
coins_dt = {}
coins_dubl = {}
for coin, paths in coins.items():
    for path in paths:
        dt = DatasetTimeseries(path)
        dt.clear_dataset()
        coins_dt.setdefault(coin, [])
        coins_dt[coin].append(dt)

    df_new = pd.concat(map(lambda dt: dt.get_dataset(), coins_dt[coin]), ignore_index=True)
    df_new = DatasetTimeseries(df_new).dataset_clear()
    df_new.drop_duplicates(subset=["datetime", "open", "min", "max", "close", "volume"], inplace=True)
    df_new = DatasetTimeseries(df_new)
    coins_dubl.setdefault(coin, len(df_new.duplicated()))

    df_new.set_dataset(df_new.clear_dataset())
    path_coin = data_manager.create_dir("processed", coin)
    df_new.set_path_save(data_manager.create_dir("processed", path_coin / "5m"))
    df_new.save_dataset(f"clear-{coin}-5m.csv")
    coins_dt[coin] = df_new
    

Function clear_dataset executed in 0:00:00.007170
Function clear_dataset executed in 0:00:00.275226
Function clear_dataset executed in 0:00:02.649099
Function clear_dataset executed in 0:00:00.019079
Function clear_dataset executed in 0:00:00.010336
Function clear_dataset executed in 0:00:00.009349
Function clear_dataset executed in 0:00:00.009133
Function clear_dataset executed in 0:00:02.682958
Directory created: /Users/igor/Desktop/App-parser/data/processed/TON
Directory created: /Users/igor/Desktop/App-parser/data/processed/TON/5m
Dataset saved to /Users/igor/Desktop/App-parser/data/processed/TON/5m/clear-TON-5m.csv


In [8]:
for coin, count in coins_dubl.items():
    if count == 0:
        continue
    print(f"Coin: {coin}, Duplicates: {count}")

In [9]:
for coin, dt in coins_dt.items():
    if (len(dt.get_dataset_Nan()) / len(dt.get_dataset())) < 0.3:
        print(f"{coin=}\n{dt.get_dataset().shape=}\n{dt.get_dataset_Nan().shape=}")
        print(len(dt.get_dataset_Nan()) / len(dt.get_dataset()))
        print()
    

coin='TON'
dt.get_dataset().shape=(100213, 6)
dt.get_dataset_Nan().shape=(749, 6)
0.007474080209154501



In [10]:
# dt_TON = coins_dt["AVAX"]
# df: pd.DataFrame = dt_TON.get_dataset()
# # dubl = df.duplicated(subset=["datetime"], keep=False)
# for data in dt_TON:
#     if data["volume"] != "x" and data["volume"] > 100:
#         print(data)
    

In [11]:
from backend.MMM import Agent, AgentPReadTime
from backend.Dataset import Indicators
from backend.train_models import Loader

In [12]:
def transform_data(bath: pd.DataFrame) -> pd.DataFrame:
    bath["SMA20"] = Indicators.sma(bath, 20)
    bath['EMA20'] = Indicators.ema(bath, 20)
    bath['RSI14'] = Indicators.rsi(bath)
    bath['MACD'], bath['Signal'] = Indicators.macd(bath)
    bath['UpperBB'], bath['MiddleBB'], bath['LowerBB'] = Indicators.bollinger_bands(bath)
    bath['ATR14'] = Indicators.atr(bath)
    bath['%K'], bath['%D'] = Indicators.stochastic_oscillator(bath)
   
    return bath
    

In [13]:
coins_dt

{'TON': <backend.Dataset.dataset.DatasetTimeseries at 0x172323f50>}

In [14]:
coin_loader = {}
for coin, dt in coins_dt.items():
    dt: DatasetTimeseries
    filter_func = lambda x: x["open"] != "x"

    loader = dt.get_time_line_loader(time_line_size=100, 
                                filter_data=filter_func)
    
    coin_loader[coin] = loader

loader_train = Loader("agent_pred_train")
agent_manager = loader_train.load_model(count_agents=1000)


/Users/igor/Library/Caches/pypoetry/virtualenvs/postgresdocker-MncL8gKv-py3.12/lib/python3.12/site-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
Loading Agent: agent_pred_train
Loading schema for: agent_pred_train


In [15]:
# agent_manager.agent[0].model

In [16]:
loader_train.train_model(list(coin_loader.values()),
                                  agent_manager=agent_manager,
                                  mixed=True)

🚀 Starting ensemble training
🏋️ Training Agent 1:   0%|          | 0/1000 [00:00<?, ?it/s]

⚠️ Mixed precision на MPS может работать некорректно. Рекомендуется использовать fp32


/Users/igor/Library/Caches/pypoetry/virtualenvs/postgresdocker-MncL8gKv-py3.12/lib/python3.12/site-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Users/igor/Desktop/App-parser/backend/train_models/loader.py:67: UserWarning: Using a target size (torch.Size([30, 5, 16])) that is different to the input size (torch.Size([30, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  price_loss = F.mse_loss(price_pred, targets)
🏋️ Training Agent 1:   0%|          | 0/1000 [00:06<?, ?it/s]


RuntimeError: The size of tensor a (5) must match the size of tensor b (16) at non-singleton dimension 2